In [1]:
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import keras
from keras import layers
from keras import models
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model

import os #システム操作系
from pathlib import Path #ファイル操作系

from scipy.optimize import curve_fit    # フィッティング用

import datetime
import time

import pandas as pd
import pickle

dt_now = datetime.datetime.now()
print('現在時刻：', dt_now)

Using TensorFlow backend.


現在時刻： 2020-12-10 20:31:10.611862


In [2]:
# このファイルの存在するフォルダの絶対パスを取得
dir_name = str(Path().resolve())
print('このファイルの存在するフォルダ：', dir_name)
# 保存先フォルダのパス作成
save_folder = os.path.join(dir_name, f'Learnings')
print('保存フォルダ：', save_folder)
# 保存先フォルダの作成(既に存在する場合は無視される)
os.makedirs(save_folder, exist_ok=True)

このファイルの存在するフォルダ： /Users/nagaiyuma/Documents/myprogram
保存フォルダ： /Users/nagaiyuma/Documents/myprogram/Learnings


In [3]:
# グラフの初期設定
plt.rcParams["figure.figsize"] = [3.14, 3.14] # 図の縦横のサイズ([横(inch),縦(inch)])
plt.rcParams["figure.dpi"] = 200 # dpi(dots per inch)
plt.rcParams["figure.facecolor"] = 'white' # 図の背景色
plt.rcParams["figure.edgecolor"] = 'black' # 枠線の色
plt.rcParams["font.family"] = "serif"       # 使用するフォント
plt.rcParams["font.serif"] = "Times New Roman"
plt.rcParams["font.size"] = 14              # 基本となるフォントの大きさ
plt.rcParams["xtick.direction"] = "in"      # 目盛り線の向き、内側"in"か外側"out"かその両方"inout"か
plt.rcParams["ytick.direction"] = "in"      # 目盛り線の向き、内側"in"か外側"out"かその両方"inout"か
plt.rcParams["xtick.bottom"] = True         # 下部に目盛り線を描くかどうか
plt.rcParams["ytick.left"] = True           # 左部に目盛り線を描くかどうか
plt.rcParams["xtick.major.size"] = 2.0      # x軸主目盛り線の長さ
plt.rcParams["ytick.major.size"] = 2.0      # y軸主目盛り線の長さ
plt.rcParams["xtick.major.width"] = 0.3     # x軸主目盛り線の線幅
plt.rcParams["ytick.major.width"] = 0.3     # y軸主目盛り線の線幅
plt.rcParams["xtick.minor.visible"] = False # x軸副目盛り線を描くかどうか
plt.rcParams["ytick.minor.visible"] = False # y軸副目盛り線を描くかどうか
plt.rcParams["xtick.minor.size"] = 2.0      # x軸副目盛り線の長さ
plt.rcParams["ytick.minor.size"] = 2.0      # y軸副目盛り線の長さ
plt.rcParams["xtick.minor.width"] = 0.3     # x軸副目盛り線の線幅
plt.rcParams["ytick.minor.width"] = 0.3     # y軸副目盛り線の線幅
plt.rcParams["xtick.labelsize"] = 8        # 目盛りのフォントサイズ
plt.rcParams["ytick.labelsize"] = 8        # 目盛りのフォントサイズ
plt.rcParams["xtick.major.pad"] = 3.0      # x軸から目盛までの距離
plt.rcParams["ytick.major.pad"] = 4.0      # y軸から目盛までの距離
plt.rcParams["axes.labelsize"] = 10         # 軸ラベルのフォントサイズ
plt.rcParams["axes.linewidth"] = 0.4        # グラフ囲う線の太さ
plt.rcParams["axes.grid"] = False           # グリッドを表示するかどうか

In [4]:
#フィッティングパラメータ取得
data_param_path = '/Users/nagaiyuma/Documents/myprogram/201209/2020-12-10_15-19_fit-param.pkl'
with open(data_param_path, mode="rb") as f:
    param = pickle.load(f)

print(param)

[ 9.28827417e-01 -1.07953945e-01  5.97782015e+02  4.59929242e+02
  5.75242649e+01  5.69547526e+01  5.18587130e-02]


In [13]:
train_num = 10000 #訓練データの数
test_num = 1000 #テストデータの数
data_size = 200 #配列の大きさ

train_x = np.zeros((train_num,data_size,data_size))
train_t = np.zeros((train_num, len(param)))
test_x = np.zeros((test_num,data_size,data_size))
test_t = np.zeros((test_num, len(param)))

In [9]:
#訓練データ作成
N = train_num #訓練データの個数

def gaussian_beam_xy(X,i0,b0,x0,y0,wx,wy,h0):
    x,y = X
    return  (i0*np.exp((-2/(1-b0**2))*(((x-x0)/wx)**2+((y-y0)/wy)**2-2*b0*((x-x0)/wx)*((y-y0)/wy)))+ h0).flatten()
#中心位置
idx = np.zeros((2, N))
idx[0] = np.arange(0.0, 1.0, 0.1).repeat(N/10)
idx = idx.T

#画像サイズ
size = 100
x_array = np.arange(0, size*2, 1.0)                         # x配列
y_array = np.arange(0, size*2, 1.0)                         # y配列
nx = len(x_array)
ny = len(y_array)
x_grid, y_grid = np.meshgrid(x_array, y_array)
intensity = np.zeros((N,nx,ny))
idx = idx + size #中心に持ってくる
start = time.time()
for n in range(N):
    #初期パラメータ
    i0 = param[0]
    b0 = param[1]
    x0 = idx[n][0]
    y0 = idx[n][1]
    wx = param[4]
    wy = param[5]
    h0 = param[6]
    param = np.array([i0, b0, x0, y0, wx, wy, h0])  #初期値
    train_t[n] = param
    
    #強度の計算
    intensity[n] = gaussian_beam_xy((x_grid.T, y_grid.T), *param).reshape(nx,ny)

intensity_noise = np.zeros((N, nx, ny))
intensity_i = np.zeros(nx*ny)
NOISE = np.zeros((N, nx*ny))

for i in range(10):
    intensity_i = intensity[int(i*N/10)].flatten()
    print(idx[int(i*N/10)])
    for k in range(nx*ny):
        NOISE[int(i*N/10):int((i+1)*N/10),k] = (np.random.normal(loc=0,scale=0.00844*intensity_i[k],size=int(N/10))+0.00165)
NOISE = NOISE.reshape(N,nx,ny)
intensity_noise = intensity.reshape(N,nx,ny) + NOISE.reshape(N,nx,ny)

train_x = intensity_noise.reshape(-1, 200, 200)

elapsed_time = time.time() - start
print ("経過時間:{0}".format(elapsed_time) + "[sec]")

[100. 100.]
[100.1 100. ]
[100.2 100. ]
[100.3 100. ]
[100.4 100. ]
[100.5 100. ]
[100.6 100. ]
[100.7 100. ]
[100.8 100. ]
[100.9 100. ]
経過時間:85.2913191318512[sec]


In [10]:
#テストデータ作成
N = test_num #訓練データの個数

def gaussian_beam_xy(X,i0,b0,x0,y0,wx,wy,h0):
    x,y = X
    return  (i0*np.exp((-2/(1-b0**2))*(((x-x0)/wx)**2+((y-y0)/wy)**2-2*b0*((x-x0)/wx)*((y-y0)/wy)))+ h0).flatten()
#中心位置
idx = np.zeros((2, N))
idx[0] = np.arange(0.0, 1.0, 0.1).repeat(N/10)
idx = idx.T

#画像サイズ
size = 100
x_array = np.arange(0, size*2, 1.0)                         # x配列
y_array = np.arange(0, size*2, 1.0)                         # y配列
nx = len(x_array)
ny = len(y_array)
x_grid, y_grid = np.meshgrid(x_array, y_array)
intensity = np.zeros((N,nx,ny))
idx = idx + size #中心に持ってくる
start = time.time()
for n in range(N):
    #初期パラメータ
    i0 = param[0]
    b0 = param[1]
    x0 = idx[n][0]
    y0 = idx[n][1]
    wx = param[4]
    wy = param[5]
    h0 = param[6]
    param = np.array([i0, b0, x0, y0, wx, wy, h0])  #初期値
    test_t[n] = param
    
    #強度の計算
    intensity[n] = gaussian_beam_xy((x_grid.T, y_grid.T), *param).reshape(nx,ny)

intensity_noise = np.zeros((N, nx, ny))
intensity_i = np.zeros(nx*ny)
NOISE = np.zeros((N, nx*ny))
for i in range(10):
    intensity_i = intensity[int(i*N/10)].flatten()
    print(idx[int(i*N/10)])
    for k in range(nx*ny):
        NOISE[int(i*N/10):int((i+1)*N/10),k] = (np.random.normal(loc=0,scale=0.00844*intensity_i[k],size=int(N/10))+0.00165)
NOISE = NOISE.reshape(N,nx,ny)
intensity_noise = intensity.reshape(N,nx,ny) + NOISE.reshape(N,nx,ny)

test_x = intensity.reshape(-1, 200, 200)

elapsed_time = time.time() - start
print ("経過時間:{0}".format(elapsed_time) + "[sec]")

[100. 100.]
[100.1 100. ]
[100.2 100. ]
[100.3 100. ]
[100.4 100. ]
[100.5 100. ]
[100.6 100. ]
[100.7 100. ]
[100.8 100. ]
[100.9 100. ]
経過時間:8.331423044204712[sec]


In [ ]:
#強度分布の表示
dt_now = datetime.datetime.now()
print('現在時刻：', dt_now)

save_name = dt_now.strftime("%Y-%m-%d_%H-%M") +'_intensity_noise-3d.png'
save_file = os.path.join(save_folder, save_name) # 保存先のファイルパス作成
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel("x[px]", fontsize=16)
ax.set_ylabel("y[px]", fontsize=16)
ax.set_zlabel("Intensity [a.u.]", fontsize=16)
plt.tick_params(labelsize=16)
ax.grid(False)
ax.scatter(x_grid.T, y_grid.T, intensity_noise[n], color='black', s=0.1)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
#fig.savefig(save_file, format="png", bbox_inches="tight")
plt.show()
print('保存ファイル名：', save_name)
print('保存ファイルパス：', save_file)
#カラーマップ表示
save_name = dt_now.strftime("%Y-%m-%d_%H-%M") +'_intensity-cmap.png'
save_file = os.path.join(save_folder, save_name) # 保存先のファイルパス作成
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
ax.set_xlabel("x[px]", fontsize=16)
ax.set_ylabel("y[px]", fontsize=16)
ax.tick_params(labelsize=16)
mappable = ax.pcolormesh(x_grid.T, y_grid.T, intensity_noise[n].reshape(nx,ny), cmap='jet', vmin=0.0, vmax=1.0)
cbar = fig.colorbar(mappable, ax=ax)
cbar.set_label("Error", fontsize=16)
cbar.ax.tick_params(labelsize=16)
#fig.savefig(save_file, format="png", bbox_inches="tight")
plt.show()
print('保存ファイル名：', save_name)
print('保存ファイルパス：', save_file)

In [22]:
#CNN3の実装
batch_size = 200  # 訓練データを200ずつのデータに分けて学習させる
epochs = 50 # 訓練データを繰り返し学習させる数
train_x = train_x.reshape(-1,200,200,1)
test_x = test_x.reshape(-1,200,200,1)
#レイヤー構造
model = Sequential()
model.add(Conv2D(100, (3,3), padding='same', input_shape=(200, 200, 1), activation='relu', kernel_initializer="he_normal"))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(Conv2D(50, (3,3), padding='same', activation='relu', kernel_initializer="he_normal"))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(Conv2D(50, (3,3), padding='same', activation='relu', kernel_initializer="he_normal"))
model.add(layers.Flatten())
model.add(Dense(1000, activation='relu', kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu', kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(len(param), activation='relu', kernel_initializer="he_normal"))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 200, 200, 100)     1000      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 100, 100, 100)     0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 100, 100, 50)      45050     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 50, 50, 50)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 50)        22550     
_________________________________________________________________
flatten_4 (Flatten)          (None, 125000)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 1000)             

In [ ]:
model.compile(loss='mse',
 optimizer='adam',
 metrics=['mae'])

callbacks = [keras.callbacks.TensorBoard(log_dir='./logs',
                            histogram_freq=1, 
                            batch_size=batch_size, 
                            write_graph=True, 
                            write_grads=True)]

history = model.fit(train_x, train_t,
 batch_size=batch_size,
 epochs=epochs,
 verbose=1,
 validation_data=(test_x, test_t))

score = model.evaluate(test_x, test_t, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Train on 10000 samples, validate on 1000 samples
Epoch 1/50


In [ ]:
#modelの保存
dt_now = datetime.datetime.now()
print('現在時刻：', dt_now)
#損失関数とmaeの推移
save_name = dt_now.strftime("%Y-%m-%d_%H-%M") +f'_noise{NOISE}.h5'
save_file = os.path.join(save_folder, save_name) # 保存先のファイルパス作成
model.save(save_file)
print('保存ファイル名：', save_name)
print('保存ファイルパス：', save_file)

#historyの保存
mae = history.history['mae']
val_mae = history.history['val_mae']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(mae) + 1)
save_data = np.array((epochs, mae, val_mae, loss, val_loss))
dt_now = datetime.datetime.now()
print('現在時刻：', dt_now)
save_name = dt_now.strftime("%Y-%m-%d_%H-%M") +f'_noise{NOISE}.json'
save_file = os.path.join(save_folder, save_name) # 保存先のファイルパス作成
hist_df = pd.DataFrame(history.history) 
with open(save_file, mode='w') as f:
    hist_df.to_json(f)
print('保存ファイル名：', save_name)
print('保存ファイルパス：', save_file)

#損失関数の推移
save_name = dt_now.strftime("%Y-%m-%d_%H-%M") +f'_{NOISE}_loss.svg'
save_file = os.path.join(save_folder, save_name) # 保存先のファイルパス作成
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(epochs, loss,  color="black", label = 'Train')
ax.scatter(epochs, val_loss,  color="red", label = 'Valdation')
ax.legend(frameon=False)
ax.set_xlabel('Epoch',fontsize=14)          # 軸ラベル
ax.set_ylabel('Loss',fontsize=14)
ax.set_ylim(0, 0.5)      # y軸の表示範囲
plt.tick_params(labelsize=14)
ax.set_aspect(1./ax.get_data_ratio()) # グラフを正方形にする
fig.savefig(save_file, format="svg", bbox_inches="tight")
fig.show()
print('保存ファイル名：', save_name)
print('保存ファイルパス：', save_file)

#maeの推移
save_name = dt_now.strftime("%Y-%m-%d_%H-%M") +f'_{NOISE}_mae.svg'
save_file = os.path.join(save_folder, save_name) # 保存先のファイルパス作成
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(epochs, mae,  color="black", label = 'Train')
ax.scatter(epochs, val_mae,  color="red", label = 'Valdation')
ax.legend(frameon=False)
ax.set_xlabel('Epoch',fontsize=14)          # 軸ラベル
ax.set_ylabel('MAE',fontsize=14)
ax.set_ylim(0, 0.5)      # y軸の表示範囲
plt.tick_params(labelsize=14)
ax.set_aspect(1./ax.get_data_ratio()) # グラフを正方形にする
fig.savefig(save_file, format="svg", bbox_inches="tight")
fig.show()
print('保存ファイル名：', save_name)
print('保存ファイルパス：', save_file)

In [5]:
N=10
idx = np.zeros((2, N))
idx[0] = np.arange(0.0, 1.0, 0.1).repeat(N/10)
idx = idx.T

[0. 0.]
